In [14]:
!pip install pyspark
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
N
Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
--2022-07-01 04:02:17--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  60.8MB/s    in 0.2s    

2022-07-01 04:02:18 (60.8 MB/s) - 

In [15]:
!./ngrok authtoken 2BBLv59UKnHDTBW37npj6qgQaqK_5PCETjQ2rJ9F8vSNsSbii

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import os
import pandas as pd
import numpy as np
import random

from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px

In [18]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, TimestampType, StructType
import pyspark.sql.functions as F

In [19]:
# Create SparkSession 
spark = SparkSession.builder \
      .master("local[4]") \
      .appName("SparkByExamples.com") \
      .config("spark.ui.port", "4050") \
      .getOrCreate()

In [20]:
# Make a tunnel to Spark UI to visualize SparkSQL progress/graphs/stuff
# PLEASE CLOSE IT AFTERWARDS IN CASE YOU EVER DECIDE TO USE THIS
def start_tunnel():
    import time
    import requests
    IPython = get_ipython()
    assert IPython
    IPython.system_raw('./ngrok http 4050 &')
    time.sleep(1)
    print(requests.get("http://localhost:4040/api/tunnels").json())

def stop_tunnel():
    IPython = get_ipython()
    assert IPython
    IPython.system_raw('pkill ngrok')

# Read the data


In [21]:
# Read the data
data = spark.read.csv('/content/drive/MyDrive/All-Subsystems-minute.csv', header=True)

# Cast all columns to Floats, except for Timestamp and DayOfWeek since they are special
types = {col: "float" for col in data.columns}
types["Timestamp"] = "timestamp"
types["DayOfWeek"] = "string"

data = data.select([F.col(c).cast(types[c]) for c in data.columns])
data.show()

# Create EVENTS view
data.createOrReplaceTempView("EVENTS")

+-------------------+---------------+-------------------------+------------------------+---------------------+----------------------------------+---------------------------------+--------------------+-------------------+--------------------+------------------------+-------------------+--------------------+-----------------------------+----------------------------+---------------------------------+-------------------------------+----------------------------+--------------------------------+------------------------------+---------------------------+-----------------------+-------------------+--------------------+----------------------+---------------------+---------------------+-----------------------+----------------------+----------------------+-----------------------+----------------------+----------------------+---------------------------+--------------------------+---------------------------+--------------------------+-------------------------+--------------------------+------------

In [22]:
data.columns

['Timestamp',
 'TimeStamp_Count',
 'DHW_ClothesWasherColdFlow',
 'DHW_ClothesWasherHotFlow',
 'DHW_DishwasherHotFlow',
 'DHW_HeatPumpWaterHeaterEnergyTotal',
 'DHW_HeatPumpWaterHeaterPowerTotal',
 'DHW_ManifoldColdFlow',
 'DHW_ManifoldHotFlow',
 'DHW_MixValveColdFlow',
 'DHW_RoomTempBasementHPWH',
 'DHW_SHWGlycolTempIn',
 'DHW_SHWGlycolTempOut',
 'DHW_SHWPumpsEnergywithStandby',
 'DHW_SHWPumpsPowerWithStandby',
 'DHW_StatusSolenoidColdKitchenSink',
 'DHW_StatusSolenoidColdMBAShower',
 'DHW_StatusSolenoidColdMBATub',
 'DHW_StatusSolenoidHotKitchenSink',
 'DHW_StatusSolenoidHotMBAShower',
 'DHW_StatusSolenoidHotMBATub',
 'DHW_WaterTempDishwasher',
 'DHW_WaterTempHPWHIn',
 'DHW_WaterTempHPWHOut',
 'DHW_WaterTempKSinkCold',
 'DHW_WaterTempKSinkHot',
 'DHW_WaterTempKSinkMix',
 'DHW_WaterTempMBAShwCold',
 'DHW_WaterTempMBAShwHot',
 'DHW_WaterTempMBAShwMix',
 'DHW_WaterTempMBATubCold',
 'DHW_WaterTempMBATubHot',
 'DHW_WaterTempMBATubMix',
 'DHW_WaterTempManifoldColdIn',
 'DHW_WaterTempManifol

### Discard columns with (a lot of) null values

In [23]:
def filter_columns(df, threshold=0.35):
    agg_df = df.select(*[(1.0 - (F.count(c) / F.count('*'))).alias(c) for c in df.columns]) # compute non-null/df.count ratio
    ratios = agg_df.collect()[0].asDict().items()
    cols_match_threshold = [ key for (key, value) in ratios if value <= threshold] # select only cols which value <= threshold
    return df.select(cols_match_threshold), ratios

In [24]:
# ignore everything before 2013-09-01 as it breaks some column's patterns
# Also ignore columns with too many nulls
NULL_THRESHOLD = 0.000035
DATE_THRESHOLD = datetime.strptime("2013-09-01", "%Y-%m-%d")

data_candidates = data.filter(F.col('Timestamp') >= DATE_THRESHOLD)
data_candidates, ratios = filter_columns(data_candidates, threshold=NULL_THRESHOLD)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
candidatos = data_candidates.columns
candidatos

In [ ]:
ratios

# Determine which columns can be easily used for testing

In [ ]:
pdf = data_candidates.toPandas()

In [ ]:
COL = "Load_2ndFloorLightsPowerUsage"
DAY = 24*60

df = pdf.iloc[0*DAY:1*DAY]
fig = px.line(df, x='Timestamp', y=COL)
fig.show()

df = pdf.iloc[0*DAY:7*DAY]
fig = px.line(df, x='Timestamp', y=COL)
fig.show()

df = pdf.iloc[0*DAY:31*DAY]
fig = px.line(df, x='Timestamp', y=COL)
fig.show()

plt.figure(figsize=(20, 8), dpi=80)
plt.grid(True)
plt.plot(pdf['Timestamp'], pdf[COL])
plt.show()
# fig = px.line(pdf, x='Timestamp', y=col)
# fig.show()

Columnas finales + filtros

In [25]:
columnas_decentes_para_testing = { 
    "DHW_HeatPumpWaterHeaterPowerTotal",
    "Load_1stFloorLightsPowerUsage",
    "Load_2ndFloorLightsEnergyUsage", 
    "Load_2ndFloorLightsPowerUsage",
    "Load_BR3PlugLoadsEnergyUsage", 
    "Load_KPlugLoadsEnergyUsage",
    "Load_LatentHeatEnergyUsage",
    "Load_OvenPowerTotal",
    "Load_RefrigeratorEnergywithStandby",
    "Load_RefrigeratorTemp",
}
columnas_temporalmente_indecentes = {
    "Load_ClothesWasherEnergywithStandby", 
    "Load_2ndFloorSensHeatPowerUsage", 
    "DHW_HeatPumpWaterHeaterEnergyTotal", 
    'Load_2ndFloorSensHeatEnergyUsage'
}
columnas_decentes = ['Timestamp'] + [c for c in columnas_decentes_para_testing if c not in columnas_temporalmente_indecentes]

In [26]:
column_filters = {
    "Load_2ndFloorLightsPowerUsage": [5, 20, 50, 90],
    "Load_KPlugLoadsEnergyUsage": [2500],
    "Load_OvenPowerTotal": [100, 250, 1000, 2000, 2100, 2200, 3000, 3500],
    "Load_1stFloorLightsPowerUsage": [0, 100, 400, 1000],
    "Load_RefrigeratorEnergywithStandby": [100],
    "Load_2ndFloorLightsEnergyUsage": [50, 100, 175, 250],
    "Load_BR3PlugLoadsEnergyUsage": [10, 40],
    "Load_LatentHeatEnergyUsage": [30, 70, 100, 130],
    "Load_RefrigeratorTemp": [50, 200, 300],
    "DHW_HeatPumpWaterHeaterPowerTotal": [300, 1000]
}

In [27]:
print(data.rdd.getNumPartitions())

16


In [28]:
data

DataFrame[Timestamp: timestamp, TimeStamp_Count: float, DHW_ClothesWasherColdFlow: float, DHW_ClothesWasherHotFlow: float, DHW_DishwasherHotFlow: float, DHW_HeatPumpWaterHeaterEnergyTotal: float, DHW_HeatPumpWaterHeaterPowerTotal: float, DHW_ManifoldColdFlow: float, DHW_ManifoldHotFlow: float, DHW_MixValveColdFlow: float, DHW_RoomTempBasementHPWH: float, DHW_SHWGlycolTempIn: float, DHW_SHWGlycolTempOut: float, DHW_SHWPumpsEnergywithStandby: float, DHW_SHWPumpsPowerWithStandby: float, DHW_StatusSolenoidColdKitchenSink: float, DHW_StatusSolenoidColdMBAShower: float, DHW_StatusSolenoidColdMBATub: float, DHW_StatusSolenoidHotKitchenSink: float, DHW_StatusSolenoidHotMBAShower: float, DHW_StatusSolenoidHotMBATub: float, DHW_WaterTempDishwasher: float, DHW_WaterTempHPWHIn: float, DHW_WaterTempHPWHOut: float, DHW_WaterTempKSinkCold: float, DHW_WaterTempKSinkHot: float, DHW_WaterTempKSinkMix: float, DHW_WaterTempMBAShwCold: float, DHW_WaterTempMBAShwHot: float, DHW_WaterTempMBAShwMix: float, DH

In [29]:
df = data.select([F.col(c) for c in columnas_decentes])

In [33]:
import pyspark.sql.functions as f
from pyspark.sql.functions import lit, udf, col, concat
from pyspark.sql import Window

from pyspark.sql.types import IntegerType, TimestampType, StringType, StructType, StructField

inf = 100000000

def get_groups_df(timestamps_df):
    w1 = Window.orderBy("Timestamp")
    df = timestamps_df\
            .withColumn("timeChange",
                        (f.col("Timestamp").cast("long") - f.lag("Timestamp").over(w1).cast("long") > 60).cast("int")
                    )\
            .fillna(True, subset=["Timestamp"])\
            .withColumn("Interval", 
                        f.sum(f.col("timeChange")).over(w1.rangeBetween(Window.unboundedPreceding, 0))
            )

    return df

def get_intervals_df(groups_df):
    w1 = Window.partitionBy("Interval").orderBy(f.col("Timestamp").cast("long").asc())
    first_timestamps_per_group = groups_df.withColumn("row",f.row_number().over(w1))\
                                    .filter(f.col("row") == 1)\
                                    .drop("row").sort(groups_df.Timestamp)\
                                    .withColumnRenamed("Timestamp", "FirstTimestamp")

    w2 = Window.partitionBy("Interval").orderBy(f.col("Timestamp").cast("long").desc())
    last_timestamps_per_group = groups_df.withColumn("row",f.row_number().over(w2))\
                                    .filter(f.col("row") == 1)\
                                    .drop("row").sort(groups_df.Timestamp)\
                                    .withColumnRenamed("Timestamp", "LastTimestamp")

    intervals_df = first_timestamps_per_group.join(last_timestamps_per_group, 
                                                   on=["Interval"])
    intervals_df = intervals_df.drop("timeChange").drop("Interval")
    intervals_df = intervals_df.orderBy(["Interval", "FirstTimestamp"])

    return intervals_df

def generate_dseq():
    emp_RDD = spark.sparkContext.emptyRDD()
    schema = StructType([
            StructField('FirstTimestamp', TimestampType(), True),
            StructField('LastTimestamp', TimestampType(), True),
            StructField('Threshold', IntegerType(), True),
            StructField("Timeseries", StringType(), True)
        ])
    total_intervals_df = spark.createDataFrame(data=emp_RDD, schema=schema).repartition(5)
    
    for i, col in enumerate(column_filters):
        print(col)
        df = data.select(['Timestamp', col])
    
        filters = [-inf] + column_filters[col] + [inf]
        print(filters)

        for left_value, right_value in zip(filters, filters[1:]):
            timestamps = spark.sql(f"SELECT Timestamp FROM EVENTS WHERE {col} >= {left_value} AND {col} < {right_value}").select('Timestamp')
            groups_df = get_groups_df(timestamps)
            intervals_df = get_intervals_df(groups_df)
            intervals_df = intervals_df.withColumn("Threshold", lit(left_value))
            intervals_df = intervals_df.withColumn("Timeseries", lit(col))
            total_intervals_df = total_intervals_df.union(intervals_df)
        if i > 2:
            break
    
    return total_intervals_df

dseq = generate_dseq()

Load_2ndFloorLightsPowerUsage
[-100000000, 5, 20, 50, 90, 100000000]
Load_KPlugLoadsEnergyUsage
[-100000000, 2500, 100000000]
Load_OvenPowerTotal
[-100000000, 100, 250, 1000, 2000, 2100, 2200, 3000, 3500, 100000000]
Load_1stFloorLightsPowerUsage
[-100000000, 0, 100, 400, 1000, 100000000]


In [35]:
dseq.columns

['FirstTimestamp', 'LastTimestamp', 'Threshold', 'Timeseries']

In [36]:
dseq = dseq.withColumn("Event", concat(col("Timeseries"), lit("_"), col("Threshold")))
dseq = dseq.drop("Timeseries", "Threshold")
dseq.show()

+-------------------+-------------------+--------------------+
|     FirstTimestamp|      LastTimestamp|               Event|
+-------------------+-------------------+--------------------+
|2013-07-01 04:02:14|2013-07-01 04:29:14|Load_2ndFloorLigh...|
|2013-07-01 04:30:15|2013-07-01 04:59:15|Load_2ndFloorLigh...|
|2013-07-01 05:00:17|2013-07-01 07:28:17|Load_2ndFloorLigh...|
|2013-07-01 07:32:17|2013-07-01 07:57:17|Load_2ndFloorLigh...|
|2013-07-01 08:01:17|2013-07-01 09:57:17|Load_2ndFloorLigh...|
|2013-07-01 10:59:17|2013-07-01 11:57:17|Load_2ndFloorLigh...|
|2013-07-01 12:23:17|2013-07-01 19:08:16|Load_2ndFloorLigh...|
|2013-07-01 19:09:21|2013-07-01 19:28:16|Load_2ndFloorLigh...|
|2013-07-01 19:33:09|2013-07-01 20:02:09|Load_2ndFloorLigh...|
|2013-07-01 20:12:42|2013-07-01 20:49:42|Load_2ndFloorLigh...|
|2013-07-01 20:58:29|2013-07-01 21:26:29|Load_2ndFloorLigh...|
|2013-07-01 21:27:30|2013-07-01 21:56:30|Load_2ndFloorLigh...|
|2013-07-01 21:57:31|2013-07-01 21:57:31|Load_2ndFloorL

In [38]:
w = Window.partitionBy("FirstTimestamp", "LastTimestamp", "Event").orderBy(f.col("FirstTimestamp").cast("long").desc())
dseq = dseq.withColumn("ID",f.row_number().over(w))

In [40]:
N = dseq.count()

In [41]:
def id_mapper(ID):
    return ID // (N//5)

id_mapper_udf = udf(lambda x: id_mapper(x), IntegerType())
dseq = dseq.withColumn("Slice", id_mapper_udf("ID")).drop("ID")

In [42]:
dseq.show()

+-------------------+-------------------+--------------------+-----+
|     FirstTimestamp|      LastTimestamp|               Event|Slice|
+-------------------+-------------------+--------------------+-----+
|2013-07-01 04:02:14|2013-07-01 04:29:14|Load_1stFloorLigh...|    0|
|2013-07-01 04:02:14|2013-07-01 04:29:14|Load_2ndFloorLigh...|    0|
|2013-07-01 04:02:14|2013-07-01 04:29:14|Load_KPlugLoadsEn...|    0|
|2013-07-01 04:02:14|2013-07-01 04:29:14|Load_OvenPowerTot...|    0|
|2013-07-01 04:30:15|2013-07-01 04:59:15|Load_1stFloorLigh...|    0|
|2013-07-01 04:30:15|2013-07-01 04:59:15|Load_2ndFloorLigh...|    0|
|2013-07-01 04:30:15|2013-07-01 04:59:15|Load_KPlugLoadsEn...|    0|
|2013-07-01 04:30:15|2013-07-01 04:59:15|Load_OvenPowerTot...|    0|
|2013-07-01 05:00:17|2013-07-01 07:28:17|Load_2ndFloorLigh...|    0|
|2013-07-01 05:00:17|2013-07-01 10:57:17|Load_1stFloorLigh...|    0|
|2013-07-01 05:00:17|2013-07-01 19:08:16|Load_KPlugLoadsEn...|    0|
|2013-07-01 05:00:17|2013-07-01 19

In [43]:
dseq.select("Event").distinct().show()

+--------------------+
|               Event|
+--------------------+
|Load_2ndFloorLigh...|
|Load_2ndFloorLigh...|
|Load_2ndFloorLigh...|
|Load_2ndFloorLigh...|
|Load_2ndFloorLigh...|
|Load_KPlugLoadsEn...|
|Load_KPlugLoadsEn...|
|Load_OvenPowerTot...|
|Load_OvenPowerTot...|
|Load_OvenPowerTot...|
|Load_OvenPowerTot...|
|Load_OvenPowerTot...|
|Load_OvenPowerTot...|
|Load_OvenPowerTot...|
|Load_OvenPowerTot...|
|Load_1stFloorLigh...|
|Load_1stFloorLigh...|
+--------------------+



In [44]:
dseq.write.option("header", True) \
.partitionBy("Slice") \
.mode("overwrite") \
.csv("DSEQ")

In [45]:
dseq

DataFrame[FirstTimestamp: timestamp, LastTimestamp: timestamp, Event: string, Slice: int]

In [72]:
dseq.rdd.getNumPartitions()

1

In [46]:
dev = dseq.rdd\
.map(lambda x: (x[2], [(x[3], (x[0], x[1]))]))\
.reduceByKey(lambda x, y: x + y)\
.toDF()

In [47]:
dev.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|Load_1stFloorLigh...|[{0, {2013-07-01 ...|
|Load_2ndFloorLigh...|[{0, {2013-07-01 ...|
|Load_KPlugLoadsEn...|[{0, {2013-07-01 ...|
|Load_OvenPowerTot...|[{0, {2013-07-01 ...|
|Load_2ndFloorLigh...|[{0, {2013-07-01 ...|
|Load_2ndFloorLigh...|[{0, {2013-07-01 ...|
|Load_2ndFloorLigh...|[{0, {2013-07-01 ...|
|Load_1stFloorLigh...|[{0, {2013-07-01 ...|
|Load_OvenPowerTot...|[{0, {2013-07-06 ...|
|Load_OvenPowerTot...|[{0, {2013-07-06 ...|
|Load_2ndFloorLigh...|[{0, {2013-07-06 ...|
|Load_OvenPowerTot...|[{0, {2013-07-14 ...|
|Load_KPlugLoadsEn...|[{0, {2013-08-05 ...|
|Load_OvenPowerTot...|[{0, {2013-08-17 ...|
|Load_OvenPowerTot...|[{0, {2013-09-15 ...|
|Load_OvenPowerTot...|[{0, {2013-10-05 ...|
|Load_OvenPowerTot...|[{0, {2014-04-20 ...|
+--------------------+--------------------+



In [71]:
dev.rdd.getNumPartitions()

1

In [69]:
def make_bitmap(x, partitions):
    bitmap = 0
    for ev_src, _ in x:
        bitmap = bitmap | (1 << ev_src)
    return bitmap

Evs = dev.count()
dev2 = dev.rdd.map(lambda x: (x[0], x[1], make_bitmap(x[1], Evs))).toDF()

In [70]:
dev2.show()

+--------------------+--------------------+---+
|                  _1|                  _2| _3|
+--------------------+--------------------+---+
|Load_1stFloorLigh...|[{0, {2013-07-01 ...|  1|
|Load_2ndFloorLigh...|[{0, {2013-07-01 ...|  1|
|Load_KPlugLoadsEn...|[{0, {2013-07-01 ...|  1|
|Load_OvenPowerTot...|[{0, {2013-07-01 ...|  1|
|Load_2ndFloorLigh...|[{0, {2013-07-01 ...|  1|
|Load_2ndFloorLigh...|[{0, {2013-07-01 ...|  1|
|Load_2ndFloorLigh...|[{0, {2013-07-01 ...|  1|
|Load_1stFloorLigh...|[{0, {2013-07-01 ...|  1|
|Load_OvenPowerTot...|[{0, {2013-07-06 ...|  1|
|Load_OvenPowerTot...|[{0, {2013-07-06 ...|  1|
|Load_2ndFloorLigh...|[{0, {2013-07-06 ...|  1|
|Load_OvenPowerTot...|[{0, {2013-07-14 ...|  1|
|Load_KPlugLoadsEn...|[{0, {2013-08-05 ...|  1|
|Load_OvenPowerTot...|[{0, {2013-08-17 ...|  1|
|Load_OvenPowerTot...|[{0, {2013-09-15 ...|  1|
|Load_OvenPowerTot...|[{0, {2013-10-05 ...|  1|
|Load_OvenPowerTot...|[{0, {2014-04-20 ...|  1|
+--------------------+------------------